In [5]:
import cv2
import numpy as np

# Load YOLO
net = cv2.dnn.readNet("C:\\Users\\jithi\\OneDrive\\Desktop\\VsCode\\capstone\\yolov3.weights", "C:\\Users\\jithi\\OneDrive\\Desktop\\VsCode\\capstone\\yolov3.cfg")
# Get the names of the output layers, instead of the indices
layer_names = net.getLayerNames()
output_layers = net.getUnconnectedOutLayersNames()

print(f'Output Layers: {output_layers}')  # Verify output layers

# Loading the classes
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

print(f'Classes Loaded: {len(classes)}')  # Verify number of classes loaded

colors = np.random.uniform(0, 255, size=(len(classes), 3))

# Define video file path
video_path = 'C:\\Users\\jithi\\OneDrive\\Desktop\\VsCode\\capstone\\motion_tracking.mp4'

# Initialize video capture
cap = cv2.VideoCapture(video_path)

if cap.isOpened():  # Verify video path
    print("Video opened successfully")
else:
    print("Error opening video")
    exit()

while True:
    # Read the frame
    ret, frame = cap.read()

    if not ret:
        break

    # Reduce frame size to speed up processing
    frame = cv2.resize(frame, (416, 416))  # Adjust size as needed

    # YOLO object detection
    height, width, channels = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.2:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                # Update our list of bounding box coordinates, confidences, and class IDs
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply non-maxima suppression to suppress weak, overlapping bounding boxes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Ensure at least one detection exists
    if len(indices) > 0:
        # Iterate over the indices we are keeping
        for i in indices.flatten():
            # Extract the bounding box coordinates
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])

            # Draw bounding box and label on the frame
            color = [int(c) for c in colors[class_ids[i]]]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            text = "{}: {:.4f}".format(classes[class_ids[i]], confidences[i])
            cv2.putText(frame, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Display the result
    cv2.imshow('Motion Detection', frame)

    # Quit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close windows
cap.release()
cv2.destroyAllWindows()

Output Layers: ('yolo_82', 'yolo_94', 'yolo_106')
Classes Loaded: 80
Video opened successfully
